In [2]:
import numpy as np

In [7]:
nSamples = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]

In [3]:
# p - persistance
# r - ready time
# d - oczekiwany termin zakończenia
# w - weight
# Cj oznacza moment zakończenia wykonywania zadania Jj w uszeregowaniu

# p r d w
# r  >= c - p

# Generate

In [50]:
for x in n:
    time = 0
    fileData = []
    fileData.append(x)
    for _ in range(x):
        p = np.random.randint(1, 10)
        r = np.random.randint(time - 5 if time - 5 > 0 else 0 ,time + 5)
        d = np.random.randint(r+p, r+p+10)
        w = np.random.randint(0,10)
        time += p
        
        fileData.append([p,r,d,w])
    with open(f'data/instances_{x}.csv', "w") as f:
        f.write(f"{str(fileData[0])}\n")
        for line in fileData[1:]:            
            f.write(f"{' '.join([str(x) for x in line])}\n")

# Verify

In [64]:
x = 50

In [47]:
data = {}
indicies = ["136805", "136792", "136683", "132231", "136730", "136682", "136764", "136782", "136723", "136778", "136309", "136718", "136315"]
for index  in indicies:
    for samples in nSamples:
        with open(f'PTSZ-instancje/{index}/out{index}_{samples}.txt', "r") as result:
            res = result.read().split('\n')
            weight = int(res[0])
            taskQueue = [int(t) for t in res[1].split(" ")[:-1]]

            with open(f'PTSZ-instancje/{index}/in{index}_{samples}.txt', "r") as instance:
                d = instance.read().split('\n')
                n = int(d[0])                
                for i,line in enumerate(d[1:]):
                    if line == '':
                        continue
                    p,r,d,w = line.split(' ')
                    p,r,d,w = int(p), int(r), int(d), int(w)
                    data[i+1] = {"p": p, "r": r, "d": d, "w": w, "c": 0 }
                    
        calculatedWeight = 0
        time = 0
        for task in range(1, samples+1):
            if time < data[task]['r']:
                time = data[task]['r']
            # add p time to current time
            time += data[task]['p']
            c = time
            data[task]['c'] = c

            # check correctness
            if not data[task]["r"] <= data[task]["c"] - data[task]["p"]: 
                print("Error")

            # verify cost
            if data[task]["c"] > data[task]["d"]:
                calculatedWeight += data[task]["w"]
        print(calculatedWeight)
        #if calculatedWeight != weight:
        #    print(f"wrong weight. Calculated: {calculatedWeight}, read: {weight}")
        #else:
        #    print(f"Weight: {calculatedWeight} confirmed")
    print()
    print()
        
        

207
482
765
944
1243
1453
1781
2028
2268
2486


97
420
1012
1877
2793
4166
5806
8022
10249
11932


219
931
2202
3871
5797
8849
11986
16473
20214
24833


244
534
844
1147
1363
1652
1860
2189
2494
2805


301
642
1307
1710
2060
2568
3216
3748
4112
4811


84
427
1093
2074
2962
4077
6055
7583
10466
12229


163
596
1257
2149
3473
4683
6318
7760
10139
13305


240
496
756
1039
1247
1543
1722
2051
2243
2452


80
141
307
360
534
638
633
817
875
1004


259
509
710
922
1177
1523
1729
2051
2174
2412


944
1555
2484
3900
4732
5199
6401
7363
8657
10173


258
539
799
1086
1368
1651
1922
2219
2484
2754


1913
3900
5800
7894
9795
12019
13870
15608
18000
19605




In [41]:
taskQueue

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49]

In [16]:
7,2,5

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138',
 '13

In [9]:
calculatedWeight = 0
time = 0
for task in taskQueue:
    if time < data[task]['r']:
        time = data[task]['r']
    # add p time to current time
    time += data[task]['p']
    c = time
    data[task]['c'] = c
    
    # check correctness
    if not data[task]["r"] <= data[task]["c"] - data[task]["p"]: 
        print("Error")
    
    # verify cost
    if data[task]["c"] > data[task]["d"]:
        calculatedWeight += data[task]["w"]
if calculatedWeight != weight:
    print(f"wrong weight. Calculated: {calculatedWeight}, read: {weight}")
else:
    print(f"Weight: {calculatedWeight} confirmed")

Weight: 0 confirmed
